In [11]:
import geohash
from datetime import datetime

def parseLine(line):
    variables = line.split("\t")
    try:
        lat = float(variables[1])
        lon = float(variables[2])
        tem = float(variables[10])
        humidity = float(variables[8])
        wind_speed = float(variables[17])
        cloud_cover = float(variables[12])   
        
        ts = int(variables[0][0: 10])
        # if you encounter a "year is out of range" error the timestamp
        # may be in milliseconds, try `ts /= 1000` in that case
        yearMonth = datetime.utcfromtimestamp(ts).strftime('%Y-%m')
        
        gh = geohash.encode(lat, lon)
        return (gh[0: 2] + '\t' + yearMonth, tem, humidity, wind_speed, cloud_cover)
    except:
        return ('', 0, 0, 0, 0)
    
#text_file = spark.read.load('hdfs://orion11:21001/3hr_sample/sampled_2015/*', format='csv', sep='\t', inferSchema=True, header=True)
text_fileCC = sc.textFile("hdfs://orion11:21001/3hr_sample/*")

# (GeoHash, wind_energy_factor, cloud_cover)
parsed_dataCC = text_fileCC \
    .map(lambda line: parseLine(line))

In [12]:
parsed_dataCC.take(5)

[('', 0, 0, 0, 0),
 ('', 0, 0, 0, 0),
 ('', 0, 0, 0, 0),
 ('', 0, 0, 0, 0),
 ('', 0, 0, 0, 0)]

In [4]:
av_data = parsed_data.map(lambda x: (x[0], (x[1], x[2], x[3], x[4]))) \
   .mapValues(lambda x: (x, 1)) \
   .reduceByKey(lambda x, y: (((x[0][0] + y[0][0]), (x[0][1] + y[0][1]), (x[0][2] + y[0][2]), (x[0][3] + y[0][3])), x[1]+y[1])) \
   .mapValues(lambda v: (v[0][0] / v[1], v[0][1] / v[1], v[0][2] / v[1], v[0][3] / v[1]))

In [5]:
#geo-time, average_temperature, average_humidity, average_wind_speed, average_cloud_cover
av_data = av_data.sortBy(lambda x: x[0], True)

In [67]:
import numpy as np
x = [1, 2, 3, 4, 5.5]
y = [3, 4 ,5, 6.1, 7.3]
pccs = np.corrcoef(x, y)
print(pccs)

[[1.         0.99844958]
 [0.99844958 1.        ]]


In [15]:
import numpy as np
a = np.array(av_data.collect())

In [16]:
#geo - raw data in time order
raw = {} 

for i in a:
    if len(i[0]) > 0:
        raw.setdefault(i[0][0: 2], []).append(i[1])



In [19]:
#geo - average_temperature in time order
temperature_array = {} 

for i in a:
    if len(i[0]) > 0:
        temperature_array.setdefault(i[0][0: 2], []).append(i[1][0])
        
#geo - average_humidity in time order
humidity_array = {} 

for i in a:
    if len(i[0]) > 0:
        humidity_array.setdefault(i[0][0: 2], []).append(i[1][1])

#geo - average_wind_speed in time order
wind_speed_array = {} 

for i in a:
    if len(i[0]) > 0:
        wind_speed_array.setdefault(i[0][0: 2], []).append(i[1][2])
        
#geo - average_cloud_cover in time order
cloud_cover_array = {} 

for i in a:
    if len(i[0]) > 0:
        cloud_cover_array.setdefault(i[0][0: 2], []).append(i[1][3])


In [22]:
import numpy as np

for key in temperature_array.keys():
    print('-----------------')
    print('geohash: ' + key)
    print('')
    
    print('humidity: ')
    pccs = np.corrcoef(temperature_array[key], humidity_array[key])
    print(pccs)
    print('')

    print('wind_speed: ')
    pccs = np.corrcoef(temperature_array[key], wind_speed_array[key])
    print(pccs)
    print('')

    print('could_cover: ')
    pccs = np.corrcoef(temperature_array[key], cloud_cover_array[key])
    print(pccs)
    print('')

-----------------
geohash: 8g

humidity: 
[[ 1.         -0.07573358]
 [-0.07573358  1.        ]]

wind_speed: 
[[ 1.        -0.0757577]
 [-0.0757577  1.       ]]

could_cover: 
[[ 1.         -0.38374192]
 [-0.38374192  1.        ]]

-----------------
geohash: 8u

humidity: 
[[ 1.         -0.26890238]
 [-0.26890238  1.        ]]

wind_speed: 
[[ 1.         -0.25467187]
 [-0.25467187  1.        ]]

could_cover: 
[[ 1.         -0.27426788]
 [-0.27426788  1.        ]]

-----------------
geohash: 8v

humidity: 
[[ 1.        -0.6180439]
 [-0.6180439  1.       ]]

wind_speed: 
[[ 1.         -0.57841825]
 [-0.57841825  1.        ]]

could_cover: 
[[ 1.         -0.61329266]
 [-0.61329266  1.        ]]

-----------------
geohash: 8x

humidity: 
[[ 1.         -0.81826103]
 [-0.81826103  1.        ]]

wind_speed: 
[[ 1.         -0.34704516]
 [-0.34704516  1.        ]]

could_cover: 
[[ 1.         -0.11887706]
 [-0.11887706  1.        ]]

-----------------
geohash: 8y

humidity: 
[[ 1.         -0.6

In [2]:
#Step1: Determine temperature trends over the past 5 years

import geohash
from datetime import datetime

def parseLine(line):
    variables = line.split("\t")
    try:
        lat = float(variables[1])
        lon = float(variables[2])
        tem = float(variables[10])
        
        ts = int(variables[0][0: 10])
        # if you encounter a "year is out of range" error the timestamp
        # may be in milliseconds, try `ts /= 1000` in that case
        year = datetime.utcfromtimestamp(ts).strftime('%Y')
        
        gh = geohash.encode(lat, lon)
        return (gh[0: 2] + '\t' + year, tem)
    except:
        return ('', 0)
    
#text_file = spark.read.load('hdfs://orion11:21001/3hr_sample/sampled_2015/*', format='csv', sep='\t', inferSchema=True, header=True)
text_fileCC = sc.textFile("hdfs://orion11:21001/3hr_sample/*")

# (GeoHash, wind_energy_factor, cloud_cover)
parsed_dataCC = text_fileCC \
    .map(lambda line: parseLine(line))

av_data = parsed_dataCC.map(lambda x: (x[0], x[1])) \
    .mapValues(lambda x: (x, 1)) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1]+y[1])) \
    .mapValues(lambda v: (v[0] / v[1])) \
    .sortBy(lambda x: x[0], True)



In [7]:
av_data.take(100)

[('', 0.0),
 ('8g\t2015', 297.690418886577),
 ('8g\t2016', 296.95031828322885),
 ('8g\t2017', 296.97685361751894),
 ('8g\t2018', 297.2065380172616),
 ('8g\t2019', 297.20918538700556),
 ('8u\t2015', 296.1241540486596),
 ('8u\t2016', 295.3445887034448),
 ('8u\t2017', 295.5010970676411),
 ('8u\t2018', 295.3283890255192),
 ('8u\t2019', 295.5756227216839),
 ('8v\t2015', 294.03979944720624),
 ('8v\t2016', 293.2754249693969),
 ('8v\t2017', 293.6035268397793),
 ('8v\t2018', 293.19427811406007),
 ('8v\t2019', 293.459893260478),
 ('8x\t2015', 286.232613257024),
 ('8x\t2016', 285.1059566207626),
 ('8x\t2017', 283.95964816102475),
 ('8x\t2018', 285.39758846153836),
 ('8x\t2019', 285.7447988339223),
 ('8y\t2015', 291.7685598813729),
 ('8y\t2016', 290.547453051791),
 ('8y\t2017', 290.7208081282063),
 ('8y\t2018', 290.48003822555637),
 ('8y\t2019', 291.31321346012226),
 ('8z\t2015', 288.91622433635393),
 ('8z\t2016', 287.3378245261116),
 ('8z\t2017', 286.5667608551815),
 ('8z\t2018', 287.094309633069

In [14]:
def parseLine(line):
    try:
        key = line[0]
        value = line[1]
        
        print(key)
        print(value)
        
        variables = key.split("\t")
        year = variables[1]
        geo = variables[0]
        
        if year == '2015':
            return (geo, value, 0, 0, 0, 0)
        elif year == '2016':
            return (geo, 0, value, 0, 0, 0)
        elif year == '2017':
            return (geo, 0, 0, value, 0, 0)
        elif year == '2018':
            return (geo, 0, 0, 0, value, 0)
        elif year == '2019':
            return (geo, 0, 0, 0, 0, value)
        else:
            return ('', 0, 0, 0, 0, 0)
    except:
        return ('', 0, 0, 0, 0, 0)
    

trend_data = av_data \
    .map(lambda line: parseLine(line)) \
    .map(lambda x: (x[0], (x[1], x[2], x[3], x[4], x[5]))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2], x[3] + y[3], x[4] + y[4]))

#year_data.take(100)

In [25]:
#Anser Question1: temperature trends over the past 5 years.

import numpy as np
a = np.array(trend_data.collect())

for i in a:
    print('-----------------')
    print('geohash: ' + i[0] + '. Five years trend: ' + str(i[1]))
    print('-----------------')

-----------------
geohash: . Five years trend: (0, 0, 0, 0, 0)
-----------------
-----------------
geohash: 9t. Five years trend: (292.3740264019511, 292.0545433190392, 292.70628247103974, 292.40777327929044, 292.17809724301713)
-----------------
-----------------
geohash: b9. Five years trend: (281.949714309276, 281.86577098414347, 280.83229689863066, 281.53038880701735, 282.3397393457142)
-----------------
-----------------
geohash: 8g. Five years trend: (297.69041888657694, 296.95031828322897, 296.9768536175187, 297.20653801726144, 297.2091853870057)
-----------------
-----------------
geohash: 9d. Five years trend: (301.2127442889999, 300.8902706132628, 300.5322136973999, 300.8009366723189, 300.3252629504864)
-----------------
-----------------
geohash: dz. Five years trend: (283.69802537820124, 283.69510325560543, 282.3962663372653, 282.60797963636367, 282.9591605380077)
-----------------
-----------------
geohash: 9x. Five years trend: (282.148811585847, 280.752185913128, 280.828